https://drops.dagstuhl.de/storage/00lipics/lipics-vol228-fscd2022/LIPIcs.FSCD.2022.27/LIPIcs.FSCD.2022.27.pdf

0 + y → y 
s(x) + y → s(x + y) 
0 × y → 0 
s(x) × y → (x × y) + y

0N = 0 
s(x) = x + 2 
+(x, y) = x + y + 2 
×(x, y) = xy + 2x + 2y + 2

http://cl-informatik.uibk.ac.at/users/thiemann/paper/SAT07Polo.pdf  SAT Solving for Termination Analysis with
Polynomial Interpretations?

use pysat?

examples from traat

Can cegis it. Maybe enumerate 0-5 possible values to start.
Or we could have some conservative prunings.

desrhowitz termination examples https://www.cs.tau.ac.il//~nachum/papers/printemp-print.pdf

 https://github.com/deividrvale/fo-termlib-python
https://github.com/hezzel/wanda https://github.com/hezzel/cora
https://smt-workshop.cs.uiowa.edu/2016/slides/fuhs.pdf SMT Techniques and Solvers in Automated Termination Analysis - Carsten Fuhs


In [ ]:
def poly_interp(vs, e, env):
    if any(v.eq(e) for v in vs):
        return smt.Int(v.decl().name())
    else:
        decl = e.decl()
        children = [poly_interp(vs, c, env) for c in e.children()]
        if decl in env:
            return env[decl](*children)
        else:
            c0 = smt.FreshConst(smt.IntSort(), prefix=decl.name())
            lincoeff = [smt.FreshConst(smt.IntSort(), prefix=decl.name()) for _ in range(len(children))]
            env[decl] = lambda *args: c0 + sum(a*b for a, b in zip(lincoeff, args))
            return env[decl](*children)

def orient(vs, lhs, rhs):
    return smt.ForAll(vs, poly_interp(lhs) > poly_interp(rhs))



s = smt.Solver()
for vs, lhs, rhs in rules:
    s.add(smt.Or(orient(vs, lhs, rhs), orient(vs, rhs, lhs)))

@functools.cache
def interp_decl(f : smt.FuncDeclRef, n):

def args(prefix, n):
    return [smt.Int(prefix + str(i)) for i in range(n)]

n = f.arity()
xs = args(n)
xs = args("x", n)
ys = args("y", n)
smt.ForAll(xs, x < y, interp_decl(f)(*xs) > interp_decl(f)(*ys))








In [3]:
from z3 import *
N = IntSort()
zero = Function("zero", N)
succ = Function("succ", N, N)
add = Function("add", N, N, N)
mul = Function("mul", N, N, N)
x,y,z = Ints('x y z')

rules = [
    (add(zero(), y), y),
    (add(succ(x), y), succ(add(x, y))),
    (mul(zero(), y), zero()),
    (mul(succ(x), y), add(mul(x, y), y))
]

interp = [
    zero() == 0,
    ForAll([x], succ(x) == x + 2),
    ForAll([x,y], add(x,y) == x + y + 2),
    ForAll([x,y], mul(x,y) == x * y + 2*x + 2*y + 2)
]


In [7]:
s = Solver()
s.add(y >= 0)
s.add(x >= 0)
for i in interp:
    s.add(i)
for lhs,rhs in rules:
    s.push()
    #s.prove(lhs > rhs)
    s.add(Not(lhs > rhs))
    if s.check() == sat:
        print(lhs,rhs)
        print(s.model())
    s.pop()

add(succ(x), y) succ(add(x, y))
[zero = 0,
 x = 8365,
 y = 0,
 add = [(8367, 0) -> 8369,
        (8365, 0) -> 8367,
        else -> 2 + Var(1) + Var(0)],
 mul = [else -> 2 + 2*Var(1) + 2*Var(0) + Var(0)*Var(1)],
 succ = [8365 -> 8367, 8367 -> 8369, else -> 2 + Var(0)]]


interp2
s(x) = x + 1 
+(x, y) = 2x + y

In [8]:
interp2 = [
    ForAll([x], succ(x) == x + 1),
    ForAll([x,y], add(x,y) == 2*x + y),
]

s = Solver()
s.add(y >= 0)
s.add(x >= 0)
for i in interp2:
    s.add(i)
for lhs,rhs in rules:
    s.push()
    #s.prove(lhs > rhs)
    s.add(Not(lhs > rhs))
    if s.check() == sat:
        print(lhs,rhs)
        print(s.model())
    s.pop()

add(zero, y) y
[zero = 0,
 x = 0,
 y = 0,
 add = [(0, 0) -> 0, else -> Var(1) + 2*Var(0)],
 succ = [else -> 1 + Var(0)]]
mul(zero, y) zero
[zero = 10450,
 x = 0,
 y = 0,
 mul = [else -> 0],
 add = [else -> Var(1) + 2*Var(0)],
 succ = [else -> 1 + Var(0)]]
mul(succ(x), y) add(mul(x, y), y)
[x = 1142,
 y = 562,
 mul = [(1143, 562) -> 0, else -> -281],
 add = [(-281, 562) -> 0, else -> Var(1) + 2*Var(0)],
 succ = [1142 -> 1143, else -> 1 + Var(0)]]
